# GO2 Learning to stand via genetic algo 

## Setup

In [12]:
import numpy as np
import mujoco
import mujoco.viewer
import mediapy as media
from robot_descriptions import go2_mj_description

### Import Model

Let's take a look at the model xml

In [10]:
with open(go2_mj_description.MJCF_PATH, 'r') as file:
    model_xml = file.read()
print(model_xml)

<mujoco model="go2">
  <compiler angle="radian" meshdir="assets" autolimits="true"/>

  <option cone="elliptic" impratio="100"/>

  <default>
    <default class="go2">
      <geom friction="0.6" margin="0.001" condim="1"/>
      <joint axis="0 1 0" damping="2" armature="0.01" frictionloss="0.2"/>
      <motor ctrlrange="-23.7 23.7"/>
      <default class="abduction">
        <joint axis="1 0 0" range="-1.0472 1.0472"/>
      </default>
      <default class="hip">
        <default class="front_hip">
          <joint range="-1.5708 3.4907"/>
        </default>
        <default class="back_hip">
          <joint range="-0.5236 4.5379"/>
        </default>
      </default>
      <default class="knee">
        <joint range="-2.7227 -0.83776"/>
        <motor ctrlrange="-45.43 45.43"/>
      </default>
      <default class="visual">
        <geom type="mesh" contype="0" conaffinity="0" group="2"/>
      </default>
      <default class="collision">
        <geom group="3"/>
        <default c

In [2]:
model = mujoco.MjModel.from_xml_path(go2_mj_description.PACKAGE_PATH + "/scene.xml")
data = mujoco.MjData(model)

renderer = mujoco.Renderer(model)

## Experimentation

First up, lets render the scene

In [13]:
duration = 3.8  # (seconds)
framerate = 60  # (Hz)

frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

This browser does not support the video tag.


Plan is to create a neural network to control the robot. 
What are the inputs/outputs?
Robot needs to be aware of state (joint angles, joint velocities)

In [15]:
joint_angles = data.qpos[:]
joint_velocities = data.qvel[:]

print("Joint Angles:", joint_angles)
print("Joint Velocities:", joint_velocities)

Joint Angles: [-2.31077083e-01  3.60823885e-05  2.01874471e-01  9.98832719e-01
  1.61972930e-03 -4.82702842e-02 -7.45659896e-04 -4.37713251e-02
 -2.47718498e-01 -1.21383487e+00  4.76953049e-02 -2.43824038e-01
 -1.22534209e+00  2.05620812e-03 -5.27488996e-01 -1.36041198e+00
 -9.89144735e-03 -5.28323549e-01 -1.38512587e+00]
Joint Velocities: [-1.97806386e-04  5.96876720e-05 -1.29046178e-04 -1.39332993e-04
 -1.54372867e-04  2.94737465e-05  3.48070711e-06 -1.18427693e-03
  1.79027743e-03 -2.26846183e-04 -1.11379865e-03  1.88247134e-03
  7.58425285e-04  1.74677307e-04 -1.60399278e-02 -8.49491203e-04
 -1.60461740e-04 -1.24000048e-02]


The network should output the torques to be applied at each joint.
Let's try to control a joint using torque control.

In [20]:
frames = []
mujoco.mj_resetData(model, data)  # Reset state and time.
while data.time < duration:
  mujoco.mj_step(model, data)
  if len(frames) < data.time * framerate:
    data.qvel[0] = 0.3
    renderer.update_scene(data)
    pixels = renderer.render()
    frames.append(pixels)
media.show_video(frames, fps=framerate)

This browser does not support the video tag.
